In [ ]:
# 1. 准备数据，将所有输入数据变为tensor
from keras.datasets import imdb
import numpy as np

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        # 将全0矩阵result(25000, 10000)中的每一行，sequence位的值由0变为1
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

x_val = x_train[:10000]
x_train_rest = x_train[10000:]

y_val = y_train[:10000]
y_train_rest = y_train[10000:]

In [ ]:
# 2. 构建网络

from keras import models, layers

model = models.Sequential()
# model.add(layers.Dense(16, activation='relu', input_shape=(10000, )))
# 编译之后提示不建议使用input_shape，而是使用Input作为第一层
model.add(layers.Input((10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# 3. 选择合适目标函数、优化器、监控指标
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 4. 进行训练

model_fit = model.fit(x_train_rest, y_train_rest, epochs=20, batch_size=512, validation_data=(x_val, y_val))

In [ ]:
# 5. 绘制训练损失和验证损失，检查是否过拟合

import matplotlib.pyplot as plt

history_dict = model_fit.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)
axs[0].plot(epochs, loss_values, 'bo', label='Training loss')
axs[0].plot(epochs, val_loss_values, 'b', label='Validation loss')
axs[0].set_title('Training and validation loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend()

axs[1].plot(epochs, acc_values, 'ro', label='Training acc')
axs[1].plot(epochs, val_acc_values, 'r', label='Validation acc')
axs[1].set_title('Training and validation acc')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Acc')
axs[1].legend()

In [ ]:
# 6. 使用大概未过拟合的epoch，来fit，最后在测试集上做evaluate
from keras import models, layers

model_fit = models.Sequential()
model_fit.add(layers.Input((10000,)))
model_fit.add(layers.Dense(16, activation='relu'))
model_fit.add(layers.Dense(16, activation='relu'))
model_fit.add(layers.Dense(1, activation='sigmoid'))
model_fit.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_fit.fit(x_train_rest, y_train_rest, epochs=5, batch_size=512, validation_data=(x_val, y_val))
results = model_fit.evaluate(x_test, y_test)
print("test loss, test acc:", results)